<a href="https://colab.research.google.com/github/ashishk121/Quantum-Hamming-distance-for-Annealer/blob/main/large%20n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
import numpy as np
from scipy import linalg
import time
import matplotlib.pyplot as plt
import math

# magic word for producing visualizations in notebook
%matplotlib inline
from collections import defaultdict
import numpy as np
from numpy import array, matrix
from scipy.io import mmread, mmwrite
import networkx as nx
import dwave_networkx as dnx
from dimod.binary_quadratic_model import BinaryQuadraticModel
from dwave.system.composites import EmbeddingComposite

import dimod
from dimod.binary_quadratic_model import BinaryQuadraticModel
from dwave.system.composites import EmbeddingComposite

from dwave.system.composites import AutoEmbeddingComposite

In [ ]:
#Code to generate a circulant matrix given the first row

import functools as ft
import more_itertools as mit

def composed_shifts(n):
    """Return a function of `n` circular shifts."""
    def f(x):    
        return ft.partial(mit.circular_shifts(x).__getitem__, n)()
    return f


In [ ]:
n=50
seed = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,1,1,1,1,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0])
t=np.zeros((n,n))
t[0] = seed;
for i in range(0,n):
    if i!=n-1:
        t[i+1]=np.array(composed_shifts(-1)(t[i]))

B=t

In [ ]:
n=int(50);
s=math.floor(math.log(n)/math.log(2))
d=n+n*s;
print(n,s,d)

In [ ]:
I=np.identity(n);
Bsq= B.dot(B);
Bsqd=np.diagonal(Bsq)

In [ ]:
M=np.zeros((d,d))
# Fill out top-left block
for i in range(0,n):
    for j in range(0,n):
        if j>i:
            M[i,j]=-2*B[i,j]+ 2*Bsq[i,j]
        elif i == j:
                M[i,j]= 1+Bsqd[i]
        else:
            M[i,j]=0

# Fill out first n rows
for p in range(2,s+2):
    for i in range(0,n):
        for j in range((p-1)*n,p*n):
            M[i,j]=(2**(p-1))*I[i,j-(p-1)*n]-(2**((p-1)+1))*B[i,j-(p-1)*n]    
         

 # Fill out the main diagonal
for i in range(n,d):
    M[i,i]=2**(2*math.floor(i/n))

 #Fill out parallel diagonals   
for q in range(1,s):
    for i in range(n,d-n):
        if i+q*n<d:
            M[i,i+q*n]=2**(1+q+2*math.floor(i/n))

In [ ]:
# In order to rule out the trivial codeword, we prepare n matrices of size d-1. These are elements of list
# The true min is  min ( initial[i] + QUBO min of list[i])

list=[];
initial=[];
v00=np.zeros((n,n-1))
v0=np.zeros((n,n))
for i in range(0,n):
   for j in range(0,n):
     if i<j:
       v0[i,j]=M[i,j]
     elif j<i:
       v0[i,j]=M[j,i]
   v00[i]=np.delete(v0[i],i)    
      
   v1=M[i][n:]
   v2=np.concatenate((v00[i],v1))
   v3=np.diag(v2)
   v4=np.delete(np.delete(M,i,0),i,1)+v3
   list.append(v4)
   initial.append(M[i,i])

In [ ]:
q0=defaultdict(int)
for u in range(0,d):
    for v in range(0,d):
        q0[(u,v)]= M[u,v];

In [ ]:
qlist=[];
for k in range(0,n):
    q=defaultdict(int)
    for u in range(0,d-1):
        for v in range(0,d-1):
            q[(u,v)]= list[k][u,v];
    qlist.append(q)            
      

In [ ]:
my_bucket = f"amazon-braket-f4679e036be0" # the name of the bucket
my_prefix = "testdata" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

In [ ]:
# session and device
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/Advantage_system1")
print('Device:', device)

In [ ]:
from dwave_qbsolv import QBSolv

In [ ]:
start = time.time()
sampler = BraketDWaveSampler(s3_folder,'arn:aws:braket:::device/qpu/d-wave/Advantage_system1')
sampler = EmbeddingComposite(sampler)
response = QBSolv().sample_qubo(qlist[0], solver=sampler)
end = time.time()
tm=end-start

In [ ]:
response.data_vectors['energy']

In [ ]:
array([-0., -0., -0., -0., -0., -0., -0., -0., -0., 38., 40., 40., 40.,
       40., 40., 40., 40., 40., 40., 40.])


In [ ]:
initial

In [ ]:
tm/60